In [1]:
import logging
import can
from time import sleep

# Initialize logging
logging.basicConfig(
    level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s"
)

# 初始化CAN总线
bus = can.interface.Bus(interface="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
########################################
# 导入机械爪和小米微电机CAN总线库
########################################
from gripper_control import GripperControl
from pcan_cybergear import CANMotorController

In [3]:
# 机械爪驱动器CAN ID为1
gripper = GripperControl(bus, addr=1)

# 修改闭环模式最大输出电压，从而调整最大输出加持力量，单位mV，最大可以是 5000mV
gripper.set_max_output_voltage(5000)

# 自动初始化爪开合位置
gripper.init_position()

# 合
# gripper.close_gripper()

# sleep(3)

# 自动调整位置以解除卡顿
# gripper.clear_clog()

# 开
# gripper.open_gripper()

In [3]:
gripper.set_max_output_voltage(5000)

In [8]:
gripper.close_gripper()

In [9]:
gripper.open_gripper()

In [38]:
import cv2
import numpy as np
from ultralytics import YOLO
from cv2 import getTickCount, getTickFrequency
import ollama
import re
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict

# 加载 YOLOv8 模型
model = YOLO("weights/yolov8s.pt")

# 获取摄像头内容
cap = cv2.VideoCapture(0)

# 全局变量
force_cache = {}  # 缓存力度值
last_query_time = {}  # 记录每个物体最后查询时间
QUERY_INTERVAL = 60  # 每个物体的查询间隔（秒）
current_force = None  # 当前使用的力度值
executor = ThreadPoolExecutor(max_workers=1)  # 用于异步执行力度查询
previous_detected_objects = set()  # 用于存储上一帧检测到的物体

# 定义我们感兴趣的类别ID列表
INTERESTED_CLASSES = {
    39: 'bottle',  # LayerD
    42: 'fork',    # LayerD
    43: 'knife',   # LayerD
    44: 'spoon',   # LayerD
    46: 'banana',  # LayerB
    47: 'apple',   # LayerC
    49: 'orange',  # LayerB
    50: 'broccoli',# LayerC
    51: 'carrot',  # LayerC
    52: 'hot dog', # LayerA
    53: 'pizza',   # LayerA
    54: 'donut',   # LayerA
    55: 'cake'     # LayerA
}

def get_force_value(object_name):
    message_content = f"""
    Based on the weight characteristics of the object '{object_name}', determine which level it belongs to and output only the corresponding numerical value. Do not include any code or text explanation, only output a single number.

    Level definitions:

    LayerA (1000): Very Light Objects
    - Examples: hot dog, pizza, donut, cake, and other soft foods
    - Weight range: 0-200 grams

    LayerB (2000): Light Objects
    - Examples: banana, orange, and other medium-hardness fruits
    - Weight range: 200-500 grams

    LayerC (3000): Medium Weight Objects
    - Examples: broccoli, carrot, apple, and other hard fruits/vegetables
    - Weight range: 500-1000 grams

    LayerD (4000): Heavy Objects
    - Examples: bottle, fork, knife, spoon, and other hard items
    - Weight range: over 1000 grams

    LayerE (5000): Unknown objects or objects with undetermined weight

    Based on the typical weight characteristics of '{object_name}', determine which level it belongs to and output only the corresponding value.
    """

    try:
        response = ollama.chat(model="llama3.2", stream=False, messages=[{
            "role": "assistant",
            "content": message_content
        }], options={"temperature": 0.5})
        content = response['message']['content']
        numbers = re.findall(r'\d+', content)
        return int(numbers[0]) if numbers else 5000
    except Exception as e:
        print(f"Force value query error: {e}")
        return 5000  # Return default value when error occurs

def update_force_value(object_name):
    """异步更新力度值的包装函数"""
    force_value = get_force_value(object_name)
    force_cache[object_name] = force_value
    last_query_time[object_name] = time.time()
    return force_value

def get_current_force(detected_objects):
    """获取当前应该使用的力度值"""
    global current_force, previous_detected_objects
    
    # 将检测到的物体转换为集合，方便比较
    detected_objects_set = set(detected_objects)
    
    # 如果检测到的物体集合与上一帧相同，直接返回当前力度值
    if detected_objects_set == previous_detected_objects:
        return current_force
    
    # 检查检测的物体与上一帧是否不同
    if detected_objects_set != previous_detected_objects:
        gripper.open_gripper()  # 检测的物体和上一帧不同时执行此行
    
    # 更新上一帧检测到的物体集合
    previous_detected_objects = detected_objects_set
    
    if not detected_objects:
        current_force = None
        return None

    current_time = time.time()
    
    for obj_name in detected_objects:
        # 如果物体不在缓存中或者超过查询间隔
        if (obj_name not in force_cache or 
            current_time - last_query_time.get(obj_name, 0) > QUERY_INTERVAL):
            # 异步更新力度值
            executor.submit(update_force_value, obj_name)
    
    # 使用缓存中的最大力度值
    max_force = max([force_cache.get(obj, 5000) for obj in detected_objects])
    
    # 检查力度值是否改变
    if max_force != current_force:
        current_force = max_force
        gripper.set_max_output_voltage(current_force)  # 当力度值改变时执行此行
        gripper.close_gripper()  # 当力度值改变时执行此行
    
    return current_force

while cap.isOpened():
    loop_start = getTickCount()
    success, frame = cap.read()
    
    if success:
        # 添加类别过滤
        results = model.predict(source=frame, classes=list(INTERESTED_CLASSES.keys()))
        result = results[0]
        boxes = result.boxes
        
        # 获取当前帧检测到的所有物体（只包含感兴趣的类别）
        detected_objects = []
        for cls_id in boxes.cls:
            cls_id = int(cls_id)
            if cls_id in INTERESTED_CLASSES:
                detected_objects.append(INTERESTED_CLASSES[cls_id])
        
        # 更新力度值
        force = get_current_force(detected_objects)
        
        # 在画面上显示检测到的物体和力度值
        annotated_frame = results[0].plot()
        
        # 计算FPS
        loop_time = getTickCount() - loop_start
        fps = int(getTickFrequency() / loop_time)
        
        # 显示信息
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(annotated_frame, f"FPS: {fps}", (10, 30), font, 1, (0, 0, 255), 2)
        if force:
            cv2.putText(annotated_frame, f"Force: {force}", (10, 70), font, 1, (0, 255, 0), 2)
        
        # 显示检测到的物体名称
        y_offset = 110
        for obj in detected_objects:
            cv2.putText(annotated_frame, f"Detected: {obj}", (10, y_offset), font, 0.7, (255, 0, 0), 2)
            y_offset += 30
        
        cv2.imshow('Object Detection', annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

executor.shutdown(wait=False)
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 bottle, 159.1ms
Speed: 9.0ms preprocess, 159.1ms inference, 213.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 10.1ms
Speed: 4.8ms preprocess, 10.1ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 20.9ms
Speed: 1.6ms preprocess, 20.9ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 19.3ms
Speed: 3.0ms preprocess, 19.3ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 15.1ms
Speed: 2.5ms preprocess, 15.1ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 11.1ms
Speed: 2.0ms preprocess, 11.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 15.0ms
Speed: 3.1ms preprocess, 15.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 19.7ms
Speed: 0.0ms preprocess, 19.7ms inference, 2.7ms postprocess per image at shape (1, 3

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
from cv2 import getTickCount, getTickFrequency
# 加载 YOLOv8 模型
model = YOLO("weights/yolov8s.pt")

# 获取摄像头内容，参数 0 表示使用默认的摄像头
cap = cv2.VideoCapture(0)

while cap.isOpened():
    loop_start = getTickCount()
    success, frame = cap.read()  # 读取摄像头的一帧图像
    if success:
        results = model.predict(source=frame) # 对当前帧进行目标检测并显示结果
        result = results[0]
        boxes = result.boxes
         # 打印检测到的对象的名称
        for cls_id in boxes.cls:
            class_name = model.names[int(cls_id)]  # 使用model.names获取类名称
            if class_name == 'book':
                gripper.close_gripper()
            elif class_name == 'bottle':
                gripper.open_gripper()
    annotated_frame = results[0].plot()
    # 中间放自己的显示程序
    loop_time = getTickCount() - loop_start
    total_time = loop_time / (getTickFrequency())
    FPS = int(1 / total_time)
    # 在图像左上角添加FPS文本
    fps_text = f"FPS: {FPS:.2f}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_color = (0, 0, 255)  # 红色
    text_position = (10, 30)  # 左上角位置

    cv2.putText(annotated_frame, fps_text, text_position, font, font_scale, text_color, font_thickness)
    cv2.imshow('img', annotated_frame)
    # 通过按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  # 释放摄像头资源
cv2.destroyAllWindows()  # 关闭OpenCV窗口


0: 480x640 1 cat, 163.1ms
Speed: 5.0ms preprocess, 163.1ms inference, 151.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.3ms
Speed: 3.0ms preprocess, 13.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.3ms
Speed: 1.1ms preprocess, 10.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 17.0ms
Speed: 3.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.9ms
Speed: 3.0ms preprocess, 12.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 4.0ms postprocess per image at shape (1, 3, 4

In [8]:
#######################################
# 控制同一条CAN总线上的小米Cybergear微电机
#######################################

# 小米微电机的CAN ID为101，上位机电脑的CAN ID为254
motor = CANMotorController(bus, motor_id=101, main_can_id=254)

# 配置电机最大输出电流
motor1.write_param_table("limit_cur", 1)

# 位置置零
motor.set_0_pos()

# 最大速度 单位：rad/s
motor.write_single_param("limit_spd", value=1)

# 位置模式
motor.set_run_mode(motor.RunModes.POSITION_MODE)

# 使能电机
motor.enable()

# 运动到指定位置 单位：rad
motor.write_single_param("loc_ref", value=0.5)

sleep(3)

# 停止电机
motor.disable()

NameError: name 'motor1' is not defined